In [13]:
import os
import pandas as pd
from googleapiclient.discovery import build


DEVELOPER_KEY = ''
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

def get_channel_id(youtube):
    # * 백종원 PAIK JONG WON 데이터 가져오기
    search_response=youtube.search().list(
        q="백종원 PAIK JONG WON", # 채널 검색
        part="snippet",
        maxResults=50,
    ).execute()

    # * 백종원 PAIK JONG WON channel_id 가져오기
    channel_id = search_response['items'][0]['snippet']['channelId']

    return channel_id

def get_channel_stats(youtube, channel_id):
    # * 통계 가져오기
    # * 'snippet': {'title': '백종원 PAIK JONG WON', 'description': '', 'customUrl': '@paik_jongwon', 'publishedAt': '2018-03-05T02:51:11Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/J3OI66Bc7T3nheyKJKAkXR6tb-_bpCsoRMTFoslOBCXI3DpVY8eFY4LZWzww3BEgkRjMOEoI=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height': 88}, 'medium': {'url': 'https://yt3.ggpht.com/J3OI66Bc7T3nheyKJKAkXR6tb-_bpCsoRMTFoslOBCXI3DpVY8eFY4LZWzww3BEgkRjMOEoI=s240-c-k-c0x00ffffff-no-rj', 'width': 240, 'height': 240}, 'high': {'url': 'https://yt3.ggpht.com/J3OI66Bc7T3nheyKJKAkXR6tb-_bpCsoRMTFoslOBCXI3DpVY8eFY4LZWzww3BEgkRjMOEoI=s800-c-k-c0x00ffffff-no-rj', 'width': 800, 'height': 800}}, 'localized': {'title': '백종원 PAIK JONG WON', 'description': ''}, 'country': 'KR'}
    # * 'contentDetails': {'relatedPlaylists': {'likes': '', 'uploads': 'UUyn-K7rZLXjGl7VXGweIlcA'}}
    # * 'statistics': {'viewCount': '884849112', 'subscriberCount': '5770000', 'hiddenSubscriberCount': False, 'videoCount': '518'}
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )

    response = request.execute()
    
    return response['items'] # * include 'snippet', 'contentDetails', 'statistics'

def get_video_list(youtube, channel_id):
    # * channel_id 의 playlists 를 maxResults 20개 가져온다.
    video_list = []
    playlists = youtube.playlists().list(
        channelId=channel_id,
        part='snippet',
        maxResults=20).execute()
    
    # * 내가 필요한 쿠킹로그 데이터는 [10] 에 위치해 있다.
    cooking_logs_id = playlists['items'][10]['id']

    # * 쿠킹로그의 리스트 (playlistId=cooking_logs_id,) 를 50개씩 가져온다.
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=cooking_logs_id,
        maxResults=50
    )
    next_page = True
    while next_page:
        response = request.execute()
        data = response['items']

        for video in data:
            video_id = video['contentDetails']['videoId']
            # * video_id 를 중복없이 video_list 에 삽입한다.
            if video_id not in video_list:
                video_list.append(video_id)

        # Do we have more pages?
        if 'nextPageToken' in response.keys():
            next_page = True
            request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId=cooking_logs_id,
                pageToken=response['nextPageToken'], # * google-api-python-client 에서 지원하는 nextPageToken
                maxResults=50
            )
        else:
            next_page = False # * 'nextPageToken' in response.keys() 가 없다면 End

    return video_list

def get_video_details(youtube, video_list):
    # * details 가 필요한 이유는 view_count, like_count, dislike_count, comment_count 를 가져와야 하기 때문
    stats_list=[]

    # Can only get 50 videos at a time.
    # * 0~50 까지의 데이터를 가져온다. 323개의 데이터 까지 6번 반복
    for i in range(0, len(video_list), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=video_list[i:i+50]
        )

        data = request.execute()
        for video in data['items']:
            title=video['snippet']['title']
            published=video['snippet']['publishedAt']
            description=video['snippet']['description']
            tag_count= len(video['snippet']["localized"])
            view_count=video['statistics'].get('viewCount',0)
            like_count=video['statistics'].get('likeCount',0)
            dislike_count=video['statistics'].get('dislikeCount',0)
            comment_count=video['statistics'].get('commentCount',0)
            stats_dict=dict(title=title, description=description, published=published, tag_count=tag_count, view_count=view_count, like_count=like_count, dislike_count=dislike_count, comment_count=comment_count)
            stats_list.append(stats_dict)

    return stats_list

def create_csv(video_data):
    df=pd.DataFrame(video_data)
    df['title_length'] = df['title'].str.len()
    df["view_count"] = pd.to_numeric(df["view_count"])
    df["like_count"] = pd.to_numeric(df["like_count"])
    df["dislike_count"] = pd.to_numeric(df["dislike_count"])
    df["comment_count"] = pd.to_numeric(df["comment_count"])
    # reaction used later add up likes + dislikes + comments
    df["reactions"] = df["like_count"] + df["dislike_count"] + df["comment_count"] + df["comment_count"]
    df.to_csv("/Users/cslee/vscode/self-dining-backend/csv/lastly.csv")
    

youtube = build (YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
channel_id = get_channel_id(youtube)

channel_stats = get_channel_stats(youtube, channel_id=channel_id)
# upload_id = channel_stats[0]['contentDetails']['relatedPlaylists']['uploads']

video_list = get_video_list(youtube, channel_id=channel_id)
video_data = get_video_details(youtube, video_list=video_list)

# create_csv(video_data=video_data)




{'kind': 'youtube#videoListResponse', 'etag': 'fyKFJi7ZR_q7WEy6R0mVx0bwWgw', 'items': [{'kind': 'youtube#video', 'etag': 'Zv4s9LHSI8zQpzitHolak08ddWY', 'id': '87zznFb0-us', 'snippet': {'publishedAt': '2023-03-23T07:00:03Z', 'channelId': 'UCyn-K7rZLXjGl7VXGweIlcA', 'title': '어묵탕 지겨우셨죠? 오늘 저녁, 빨간 어묵으로 끝내드릴게요!', 'description': '#백종원 #쿠킹로그 #빨간어묵\n\n어묵으로 할 수 있는 그럴싸한 요리 찾고 계세요?\n어묵탕은 지난주에 해 먹었고, 어묵볶음은 이제 질린다구요?\n그런 팀원들을 위해 준비했습니다!\n들어는 보셨나요? 치명적인 매력의 빨간 어묵!\n\n분식집에서 파는 것만 먹어도 맛있는데\n2/2/2의 공식 하나면 어렵지 않게 만들 수 있어요!\n항상 그랬듯이 우리는 간단하고 빠른 거 좋아하잖아요?\n아직은 쌀쌀한 저녁, 얼른 빨간 어묵에 소주 한잔 하자구요!\n\n[ 빨간어묵 ] 약 3~4인분\n[ 재료 ]\n정수물 1병(500ml)\n사각어묵 10장(350g)\n대파 약1대(70g)\n양파 1/4개(60g)\n청양고추 2개(20g)\n황설탕 2큰술(24g)\n고추장 2큰술(40g)\n진간장 2큰술(20g)\n굵은고추가루 1/2큰술(4g)\n고운고추가루 1/2큰술(3g)\n후추가루 약간\n\n[ 만드는 법 ]\n1. 사각어묵 5장은 길게 반 접고 다시 반 접어 4겹을 만든 후 어묵의 양끝 1cm 정도를 남기고 꼬치로 지그재그로 꽂는다.\n   *어묵을 꼬아서 꽂을 경우 길게 3번만 접어 꽂는다.\n   *꼬치가 없다면 나무젓가락을 이용해도 좋다.\n2. 나머지 사각어묵 5장은 먹기 좋은 크기로 자유롭게 잘라 준비한다.\n3. 대파는 길게 반 갈라 3cm 길이로 자르고 양파는 0.8cm 두께로 굵게 채